# Binning Data for Report 2

Testing code that allows us to find a 3 day moving average of daily average temperatures and comparing a daily average temperature to the average daily temperatures for the 3 previous days.

In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot_date
import seaborn as sns

import datetime as dt
from datetime import datetime
from IPython.display import display

In [2]:
stations = {'seattle': 'WBAN:24233', 
            'boston': 'WBAN:14739', 
            'lexington': 'WBAN:93820', 
            'san diego': 'WBAN:93107'}

# Make a date parsing function with the correct format
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M')

# Read the CSV while parsing the dates
df = pd.read_csv("913023.csv", parse_dates=['DATE'], date_parser=dateparse)

/home/william/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,8,10,11,12,13,14,15,16,17,19,20,22,23,25,26,27,28,29,33,34,37,38,39,40,44,45,46,47,48,50,59,60,62,63,64,67,68,69,71,72,73,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#Remove all columns except for STATION, DATE, DAILYAverageDryBulbTemp, DAILYPrecip
#Make a copy of the original DataFrame
seatac = df.copy(deep=True)

#Uses DATE column as row index for resampling purposes later
seatac.set_index('DATE', inplace=True)

#Select rows of data for Seattle (WBAN:24233)
seatac = seatac.loc[seatac['STATION'] == stations['seattle']]

#Select columns with data we need
seatac = seatac[['STATION', 'DAILYAverageDryBulbTemp', 'DAILYPrecip']]

#Select 2015 data
seatac_2015 = seatac.copy(deep=True)
seatac_2015 = seatac_2015.ix['2015-01-01':'2015-12-31']
display(seatac_2015)

,STATION,DAILYAverageDryBulbTemp,DAILYPrecip
DATE,,,
2015-01-01 00:53:00,WBAN:24233,NaN,NaN
2015-01-01 01:53:00,WBAN:24233,NaN,NaN
2015-01-01 02:53:00,WBAN:24233,NaN,NaN
2015-01-01 03:53:00,WBAN:24233,NaN,NaN
2015-01-01 04:00:00,WBAN:24233,34,NaN
2015-01-01 04:53:00,WBAN:24233,NaN,NaN
2015-01-01 05:53:00,WBAN:24233,NaN,NaN
2015-01-01 06:53:00,WBAN:24233,NaN,NaN
2015-01-01 07:53:00,WBAN:24233,NaN,NaN


In [6]:
# Turn all non-numerical values into NaN for columns that should be numbers (via coerce) 
seatac_2015['DAILYAverageDryBulbTemp'] = seatac_2015['DAILYAverageDryBulbTemp'].apply(pd.to_numeric, errors='coerce')
seatac_2015['DAILYPrecip'] = seatac_2015['DAILYPrecip'].apply(pd.to_numeric, errors='coerce')

#Drop rows where there is no DAILYAverageDryBulbTemp value
seatac_2015.dropna(subset=['DAILYAverageDryBulbTemp'], inplace=True)

# Replace NaN for precipitation with 0
seatac_2015['DAILYPrecip'].fillna(0, inplace=True)

# Replace trace amounts of precipitation ('T') with 0.00001
seatac_2015['DAILYPrecip'].replace('T', 0.00001, inplace=True)

# Downsamples data to 1 day -- temperatures are averaged and precipitation is added
seatac_2015 = seatac_2015.resample('1D').agg({'DAILYAverageDryBulbTemp': np.mean, 'DAILYPrecip': np.sum})

# Removes pandas upsampling that inserts NaNs for rows of days that did not previously exist
seatac_2015.dropna(subset=['DAILYAverageDryBulbTemp'], inplace=True)

display(seatac_2015)

,DAILYPrecip,DAILYAverageDryBulbTemp
DATE,,
2015-01-01,0.00,34.250000
2015-01-02,0.06,36.750000
2015-01-03,0.06,38.000000
2015-01-04,0.40,43.000000
2015-01-05,0.70,51.250000
2015-01-06,0.24,48.750000
2015-01-07,0.00,45.000000
2015-01-08,0.00,40.750000
2015-01-09,0.01,42.500000


In [ ]:
pd.rolling_mean(seatac_2015, window=3, min_periods=1)